- `print()`는 느리다. 입력이 많을것으로 예상되는 경우, `input=sys.stdin.readline`을 사용한다.
  - 메모리가 문제가 되지 않는 경우 `input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline`을 사용할 수 있다.
    - standard input으로 입력 파일의 크기(`os.fstat(0).st_size`)만큼 까지 file descriptor에서 직접 bytes로 읽어들인다. 로우 레벨에서 작동하기에 더 빠르다.
    - 읽어들인 모든 input이 메모리에 저장되므로 주의.
    - `~.readline`은 메모리에 있는 파일에서 한줄만큼의 스트림을 읽어서 반환한다.
    - `sys.stdin.readline`과 달리 개행문자를 포함한 한줄을 bytes형태로 불러온다.
      - 이에따라 특정 인코딩 입력에 대해 문제가 될 수 있으며, string을 비교해야 할 경우 `b"string"`과 같이 bytes로 비교해야 한다.

- 전역변수에 접근하는 시간은 로컬변수에 접근하는 시간보다 유의미하게 느리다.
  - 따라서 가능한 경우, 전역 범위에 변수를 선언하는것보단, \
  문제를 푸는 함수(`def sol(): ...`)를 선언해서 그 TC 안에서 필요한 변수들을 사용하는 것이 좋다.
  


- 파이썬의 index 접근은 생각보다 느리다. 느린 $O(1)$이다.
  - 예를들어 list `l`을 index인 `i`로 접근하려 할 때, \
  `for i in range(len(l)): l[i]`보다, `for v in l: v`가 더 빠르며, \
  index가 필요할 경우 `for i, v in enumerate(l): i, v`를 사용하는 것이 더 빠르다.
  - 특히 2차원 이상의 배열에서 (바뀌지 않을)원본 데이터를 읽어야 할 경우, `range`를 두번 써서 인덱스 접근을 하게 되는 경우 \
  예를들어,  `for i in range(N): for j in range(N): L[i][j]` 인터프리터는 L[i][j]의 값을 알아내기 위해 매번 `L[i][j]`에 접근해야 한다. \
  `for i, l in enumerate(L): for j, v in enumerate(l): v`의 경우 v의 값은 캐싱되어 있으므로 상대적으로 더 느린 인덱스 접근을 할 필요가 없어진다. \
  이를 통해 평균 약 10%정도 동작속도가 빨라진다.
  - 하지만 list의 길이를 동적으로 두어 append 하는 것(`l.append(x)`)보다, \
  i가 충분히 클 경우 l의 길이를 fixed 해놓고, `l[i] = x`를 사용하는 것이 더 빠르다.
    - https://www.acmicpc.net/source/61132478 `l[i] = x` 구현. 492ms
    - https://www.acmicpc.net/source/61132338 `l.append(x)` 구현. 540ms
  - DP에 저장할 값을 tuple이나 list로 하는 것은 속도를 느리게 할 수 있다. 9019(DSLR) 참고
    - 값이 2개 이상 필요한 경우 DP 테이블을 하나 더 만들어서 각각 저장하는 것이 더 빠르다.


- Python의 String은 immutable 하기 때문에, 합쳐진 문자열을 만들기 위해선 원본 문자열들의 모든 문자를 복사하는 작업을 해야 하므로 $\Omicron(N^2)$ 만큼의 시간이 소요된다.
  - 쉽게 생각해서, `string` += `string` 같은 표현을 절대 삼가자. 이 과정이 $\Omicron(N)$ 이다. \
  대신 `string`을 list로 관리하고, 그 list에 `append()`하는 방식을 사용하자. 나중에 문자열을 합칠 때 `join()`을 사용해서 한꺼번에 하는 것이 효율적이다.


- list(deque)나 tuple형태로 저장하고 꺼내쓰는 방식은 매우 비효율적이고, 느리다.
  - Python은 추가 메모리 할당을 최소화 하기 위해 메모리를 넉넉하게 잡아둔다. 반복문으로 많은 원소를 append를 하게 되면 결국 오래 걸리는 추가 메모리 할당을 해야 하며, 이는 런타임에 진행된다.
    - Python이 메모리를 필요 이상으로 사용하는 것도 이러한 이유
  - list를 만드는 작업 자체도 오래 걸린다. immutable한 값을 그대로 넣는 것보다, 오래걸리는 메모리 할당을 필요로 하는 게 당연히 시간이 더 걸린다.
  - Garbage Collector도 문제가 된다. q에서 list가 빠져나왔을 때, GC는 빠져나온 list를 할당 해제 할지의 여부를 런타임에 확인한다.
    - 즉, (확인하는 시간) + (할당 해제하는 시간)이 런타임에 소모된다.
  - 따라서 가능하면 특히, node는 list로 저장하는 것을 피해야 한다. 분리할 수 있으면 각각 나눠서 처리하자.


- Python은 재귀 함수 최적화가 없다. 흔히 몇몇 언어에서 `tail-call optimization`이 있어 재귀함수를 사용해도 빠른 경우가 있는데, `Python은 그것이 없고` 구현 예정도 없다고 한다.
  - 가능하면 iterative한 방법으로 구현하는 것이 좋지만, 재귀에 필요한 정보들을 모두 iterative하게 바꾸기는 어려운 경우가 많다. 그냥 효율적으로 잘 짜는 방법을 찾는 수 밖에 없다.
  - 추가로 재귀함수의 argument가 지나치게 많을 경우(그래프 연결정보 `G`, 그래프 길이 `N` 등등), 메모리 사용량이 문제가 될 수 있다. \
  따라서 템플릿을 잘 짜는것이 중요하다. 위와같이 변하지 않을 정보들은 그 재귀함수의 outer scope에서 참조하도록 설계해야 할 것이다. 물론 그 참조자체가 느린 global scope에선 피하고.
    - Python은 함수 호출 시 argument가 immutable하던, mutable(이 경우엔 당연히 그 데이터의 포인터)하던 `복사`해서 넘기기 때문에, 재귀함수를 사용할 때는 argument의 개수를 최소화 하는 것이 좋다. 특히 기본 recursion limit이 1000으로 돼있는 기본값을 밥먹듯이 뚫고 쓰게 될 경우엔 더더욱 그렇다.

### 시간복잡도 참고자료
- https://wiki.python.org/moin/TimeComplexity